# Create Solr Core

In [93]:
# Delete any existing core
!curl -X GET "http://localhost:8983/solr/admin/cores?action=UNLOAD&core=search_reddit&deleteInstanceDir=true&deleteDataDir=true"

{
  "responseHeader":{
    "status":400,
    "QTime":0
  },
  "error":{
    "metadata":["error-class","org.apache.solr.common.SolrException","root-error-class","org.apache.solr.common.SolrException"],
    "msg":"Cannot unload non-existent core [search_reddit]",
    "code":400
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   282  100   282    0     0   1345      0 --:--:-- --:--:-- --:--:--  1342


In [94]:
import shutil

# Create directory
shutil.copytree("C:\\solr-9.5.0-slim\\server\\solr\\configsets\\_default", "C:\\solr-9.5.0-slim\\server\\solr\\search_reddit")

'C:\\solr-9.5.0-slim\\server\\solr\\search_reddit'

In [95]:
# Create solr core with default settings
!curl -X GET "http://localhost:8983/solr/admin/cores?action=CREATE&name=search_reddit&instanceDir=search_reddit"

{
  "responseHeader":{
    "status":0,
    "QTime":500
  },
  "core":"search_reddit"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    86  100    86    0     0    121      0 --:--:-- --:--:-- --:--:--   121


# Copy the default stopwords and synonyms into created core

In [96]:
# Copy txt file and rename
shutil.copy("C:\\solr-9.5.0-slim\\server\\solr\\search_reddit\\conf\\lang\\stopwords_en.txt", "C:\\solr-9.5.0-slim\\server\\solr\\search_reddit\\conf\\stopwords.txt")


# Get NLTK stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

# Read existing stopwords from file, skip lines starting with #
existing_stopwords = []
with open("C:\\solr-9.5.0-slim\\server\\solr\\search_reddit\\conf\\stopwords.txt", "r") as f:
    for line in f:
        if not line.startswith("#"):
            existing_stopwords.append(line.strip())

# Add NLTK stopwords to existing stopwords
stopwords = set(stopwords).union(set(existing_stopwords))
# Remove duplicates
stopwords = list(set(stopwords))

# Write back to file
with open("C:\\solr-9.5.0-slim\\server\\solr\\search_reddit\\conf\\stopwords.txt", "w") as f:
    f.write("\n".join(stopwords))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aozy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [97]:
# Add custom synonyms into synonyms.txt
# Replace model names with their full brand names as well, Electric car domain specific abbreviations & slangs
mapping_dict = {
    'BMW': 'BMW I3',
    'I3': 'BMW I3',
    
    'leaf': 'Nissan Leaf',
    'Nissan': 'Nissan Leaf',
    
    'Bolt': 'Chevrolet Bolt',
    'Chevy': 'Chevrolet Bolt',
    'BoltEV': 'Chevrolet Bolt',
    
    'F150Lightning': 'Ford F150Lightning',
    'F150': 'Ford F150Lightning',
    'Ford': 'Ford F150Lightning',
    'Lightning': 'Ford F150Lightning',
    
    'Taycan': 'Porsche Taycan',
    'Porsche': 'Porsche Taycan',
    
    'EV6': 'Kia EV6',
    'Kia': 'Kia EV6',
}

# Append to synonyms.txt with this format
# pixima => pixma

with open("C:\\solr-9.5.0-slim\\server\\solr\\search_reddit\\conf\\synonyms.txt", "a") as f:
    for k, v in mapping_dict.items():
        f.write(f"{k} => {v}\n")

In [98]:
# Restart core to apply changes
!curl -X POST "http://localhost:8983/solr/admin/cores?action=RELOAD&core=search_reddit&wt=json"

{

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    60  100    60    0     0    132      0 --:--:-- --:--:-- --:--:--   131
100    60  100    60    0     0    132      0 --:--:-- --:--:-- --:--:--   131



  "responseHeader":{
    "status":0,
    "QTime":238
  }
}


# Solr recommends we use HTTP requests to edit schema. Because of data integrity issues

In [99]:
# Get current schema, should be default/reserved fields
!curl -X GET "http://localhost:8983/solr/search_reddit/schema/fields"

{
  "responseHeader":{
    "status":0,
    "QTime":0
  },
  "fields":[{
    "name":"_nest_path_",
    "type":"_nest_path_"
  },{
    "name":"_root_",
    "type":"string",
    "indexed":true,
    "stored":false,
    "docValues":false
  },{
    "name":"_text_",
    "type":"text_general",
    "multiValued":true,
    "indexed":true,
    "stored":false
  },{
    "name":"_version_",
    "type":"plong",
    "indexed":false,
    "stored":false
  },{
    "name":"id",
    "type":"string",
    "multiValued":false,
    "indexed":true,
    "required":true,
    "stored":true
  }]
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   574  100   574    0     0   2628      0 --:--:-- --:--:-- --:--:--  2621


# Add custom schema & index

https://solr.apache.org/guide/6_6/field-properties-by-use-case.html#field-properties-by-use-case
https://solr.apache.org/guide/6_6/field-type-definitions-and-properties.html#field-type-definitions-and-properties

## With only basic cleaning done beforehand, the rest have to be done in Solr. Therefore we define a custom field type for text

- https://solr.apache.org/guide/6_6/filter-descriptions.html

text_general for reference
```
</fieldType>
  <fieldType name="text_general" class="solr.TextField" positionIncrementGap="100" multiValued="true">
    <analyzer type="index">
      <tokenizer name="standard"/>
      <filter words="stopwords.txt" name="stop" ignoreCase="true"/>
      <filter name="lowercase"/>
    </analyzer>
    <analyzer type="query">
      <tokenizer name="standard"/>
      <filter words="stopwords.txt" name="stop" ignoreCase="true"/>
      <filter ignoreCase="true" synonyms="synonyms.txt" name="synonymGraph" expand="true"/>
      <filter name="lowercase"/>
    </analyzer>
</fieldType>
```

text_reddit custom field type
- Query and index filters do not need to be the same
- Synonyms are used to expand search results for query terms only
- NGram filter is used to create n-grams for index terms only
- The rest can be the same as text_general
```
  <fieldType name="text_reddit" class="solr.TextField" positionIncrementGap="100" multiValued="true">
    <analyzer type="index">
      <tokenizer class="solr.WhitespaceTokenizerFactory"/>
      <filter class="solr.StopFilterFactory" words="stopwords.txt" ignoreCase="true"/>
      <filter class="solr.LowerCaseFilterFactory"/>
      <filter class="solr.SnowballPorterFilterFactory" language="English"/>
      <filter class="solr.NGramFilterFactory" maxGramSize="20" minGramSize="2"/>
    </analyzer>
    <analyzer type="query">
      <tokenizer class="solr.WhitespaceTokenizerFactory"/>
      <filter class="solr.StopFilterFactory" words="stopwords.txt" ignoreCase="true"/>
      <filter class="solr.SynonymGraphFilterFactory" ignoreCase="true" synonyms="synonyms.txt" expand="true"/>
      <filter class="solr.LowerCaseFilterFactory"/>
      <filter class="solr.SnowballPorterFilterFactory" language="English"/>
    </analyzer>
```

In [100]:
# Tried with General tokenizer but didnt work
# !curl -X POST -H "Content-type:application/json" --data-binary "{"add-field-type":{"name":"text_reddit","class":"solr.TextField","positionIncrementGap":"100","multiValued":true,"indexAnalyzer":{"tokenizer":{"class":"solr.StandardTokenizerFactory"},"filters":[{"class":"solr.StopFilterFactory","ignoreCase":true,"words":"stopwords.txt"},{"class":"solr.LowerCaseFilterFactory"},{"class":"solr.SnowballPorterFilterFactory","language":"English"},{"class":"solr.NGramFilterFactory","minGramSize":2,"maxGramSize":20}]},"queryAnalyzer":{"tokenizer":{"class":"solr.StandardTokenizerFactory"},"filters":[{"class":"solr.StopFilterFactory","ignoreCase":true,"words":"stopwords.txt"},{"class":"solr.SynonymGraphFilterFactory","ignoreCase":true,"synonyms":"synonyms.txt","expand":true},{"class":"solr.LowerCaseFilterFactory"},{"class":"solr.SnowballPorterFilterFactory","language":"English"}]}}}" http://localhost:8983/api/cores/search_reddit/schema

# Try with Whitespace tokenizer
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field-type":{"name":"text_reddit","class":"solr.TextField","positionIncrementGap":"100","multiValued":true,"indexAnalyzer":{"tokenizer":{"class":"solr.WhitespaceTokenizerFactory"},"filters":[{"class":"solr.StopFilterFactory","ignoreCase":true,"words":"stopwords.txt"},{"class":"solr.LowerCaseFilterFactory"},{"class":"solr.SnowballPorterFilterFactory","language":"English"},{"class":"solr.NGramFilterFactory","minGramSize":2,"maxGramSize":20}]},"queryAnalyzer":{"tokenizer":{"class":"solr.WhitespaceTokenizerFactory"},"filters":[{"class":"solr.StopFilterFactory","ignoreCase":true,"words":"stopwords.txt"},{"class":"solr.SynonymGraphFilterFactory","ignoreCase":true,"synonyms":"synonyms.txt","expand":true},{"class":"solr.LowerCaseFilterFactory"},{"class":"solr.SnowballPorterFilterFactory","language":"English"}]}}}" http://localhost:8983/api/cores/search_reddit/schema

{
  "responseHeader":{
    "status":0,
    "QTime":155
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   782  100    60  100   722    158   1912 --:--:-- --:--:-- --:--:--  2074


## Field types explained
- indexed -> If true, the value of the field can be used in queries to retrieve matching documents.
- stored -> If true, the actual value of the field can be retrieved by queries.

Our search engine primarily searches by comment & post texts

```
<field name="author" type="text_general"/>
<field name="created_utc" type="pdates"/>
<field name="edited" type="pdates"/>
<field name="id" type="string" multiValued="false" indexed="true" required="true" stored="true"/>
<field name="num_comments" type="pdoubles"/>
<field name="permalink" type="text_general"/>
<field name="post_id" type="text_general"/>
<field name="score" type="plongs"/>
<field name="subreddit_name" type="text_general"/>
<field name="text" type="text_general"/>
<field name="type" type="text_general"/>
<field name="upvote_ratio" type="pdoubles"/>
<field name="url" type="text_general"/>
```

Our schema shall be:

```
<field name="author" type="string" multiValued="false" indexed="true" required="true" stored="true"/>
<field name="created_utc" type="pdates" multiValued="false" indexed="true" stored="true"/>
<field name="edited" type="pdates" indexed="true" stored="true"/>
<field name="id" type="string" multiValued="false" indexed="true" required="true" stored="true"/>
<field name="num_comments" type="pdoubles" indexed="true" stored="true"/>
<field name="permalink" type="string" stored="true"/>
<field name="post_id" type="string" stored="true"/>
<field name="score" type="plongs" indexed="true" stored="true"/>
<field name="subreddit_name" type="string" required="true" stored="true"/>
<field name="text" type="text_reddit" multiValued="true" indexed="true" required="true" stored="true"/>
<field name="type" type="string" required="true" stored="true"/>
<field name="upvote_ratio" type="pdoubles" indexed="true" stored="true"/>
<field name="url" type="string" stored="true"/>
```

In [101]:
# author 
!curl -X POST -H "Content-type:application/json" --data-binary "{\"add-field\":{\"name\":\"author\",\"type\":\"string\",\"stored\":true,\"indexed\":true,\"required\":true,\"multiValued\":false}}" http://localhost:8983/api/cores/search_reddit/schema

# created_utc
!curl -X POST -H "Content-type:application/json" --data-binary "{\"add-field\":{\"name\":\"created_utc\",\"type\":\"pdates\",\"stored\":true,\"indexed\":true,\"multiValued\":false}}" http://localhost:8983/api/cores/search_reddit/schema

# edited
!curl -X POST -H "Content-type:application/json" --data-binary "{\"add-field\":{\"name\":\"edited\",\"type\":\"pdates\",\"stored\":true,\"indexed\":true}}" http://localhost:8983/api/cores/search_reddit/schema

# id
# !curl -X POST -H "Content-type:application/json" --data-binary "{\"add-field\":{\"name\":\"id\",\"type\":\"string\",\"stored\":true,\"indexed\":true,\"required\":true,\"multiValued\":false}}" http://localhost:8983/api/cores/search_reddit/schema

# num_comments
!curl -X POST -H "Content-type:application/json" --data-binary "{\"add-field\":{\"name\":\"num_comments\",\"type\":\"pdoubles\",\"stored\":true,\"indexed\":true}}" http://localhost:8983/api/cores/search_reddit/schema

# permalink
!curl -X POST -H "Content-type:application/json" --data-binary "{\"add-field\":{\"name\":\"permalink\",\"type\":\"string\",\"stored\":true}}" http://localhost:8983/api/cores/search_reddit/schema

# post_id
!curl -X POST -H "Content-type:application/json" --data-binary "{\"add-field\":{\"name\":\"post_id\",\"type\":\"string\",\"stored\":true}}" http://localhost:8983/api/cores/search_reddit/schema

# score -> changed to upvote
!curl -X POST -H "Content-type:application/json" --data-binary "{\"add-field\":{\"name\":\"upvote\",\"type\":\"plongs\",\"stored\":true,\"indexed\":true}}" http://localhost:8983/api/cores/search_reddit/schema

# subreddit_name
!curl -X POST -H "Content-type:application/json" --data-binary "{\"add-field\":{\"name\":\"subreddit_name\",\"type\":\"string\",\"stored\":true,\"required\":true}}" http://localhost:8983/api/cores/search_reddit/schema

# text
!curl -X POST -H "Content-type:application/json" --data-binary "{\"add-field\":{\"name\":\"text\",\"type\":\"text_reddit\",\"stored\":true,\"indexed\":true,\"required\":true,\"multiValued\":true}}" http://localhost:8983/api/cores/search_reddit/schema

# type
!curl -X POST -H "Content-type:application/json" --data-binary "{\"add-field\":{\"name\":\"type\",\"type\":\"string\",\"stored\":true,\"required\":true}}" http://localhost:8983/api/cores/search_reddit/schema

# upvote_ratio
!curl -X POST -H "Content-type:application/json" --data-binary "{\"add-field\":{\"name\":\"upvote_ratio\",\"type\":\"pdoubles\",\"stored\":true,\"indexed\":true}}" http://localhost:8983/api/cores/search_reddit/schema

# url
!curl -X POST -H "Content-type:application/json" --data-binary "{\"add-field\":{\"name\":\"url\",\"type\":\"string\",\"stored\":true}}" http://localhost:8983/api/cores/search_reddit/schema

{
  "responseHeader":{
    "status":0,
    "QTime":145
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   172  100    60  100   112    162    304 --:--:-- --:--:-- --:--:--   467


{
  "responseHeader":{
    "status":0,
    "QTime":105
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   161  100    60  100   101    188    317 --:--:-- --:--:-- --:--:--   506


{
  "responseHeader":{
    "status":0,
    "QTime":100
  }
}


...

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   134  100    59  100    75    192    245 --:--:-- --:--:-- --:--:--   437


{
  "responseHeader":{
    "status":0,
    "QTime":87
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   143  100    59  100    84    195    278 --:--:-- --:--:-- --:--:--   475


{
  "responseHeader":{
    "status":0,
    "QTime":91
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   117  100    59  100    58    187    184 --:--:-- --:--:-- --:--:--   371


In [102]:
# Get current schema, should be default/reserved fields
!curl -X GET "http://localhost:8983/solr/search_reddit/schema/fields"

{
  "responseHeader":{
    "status":0,
    "QTime":0
  },
  "fields":[{
    "name":"_nest_path_",
    "type":"_nest_path_"
  },{
    "name":"_root_",
    "type":"string",
    "indexed":true,
    "stored":false,
    "docValues":false
  },{
    "name":"_text_",
    "type":"text_general",
    "multiValued":true,
    "indexed":true,
    "stored":false
  },{
    "name":"_version_",
    "type":"plong",
    "indexed":false,
    "stored":false
  },{
    "name":"author",
    "type":"string",
    "multiValued":false,
    "indexed":true,
    "required":true,
    "stored":true
  },{
    "name":"created_utc",
    "type":"pdates",
    "multiValued":false,
    "indexed":true,
    "stored":true
  },{
    "name":"edited",
    "type":"pdates",
    "indexed":true,
    "stored":true
  },{
    "name":"id",
    "type":"string",
    "multiValued":false,
    "indexed":true,
    "required":true,
    "stored":true
  },{
    "name":"num_comments",
    "type":"pdoubles",
    "indexed":true,
    "stored":true
  },

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1695  100  1695    0     0   8193      0 --:--:-- --:--:-- --:--:--  8228


# Add spellcheck suggestions in response if query returns less results

- https://solr.apache.org/guide/8_1/spell-checking.html

DirectSolrSpellChecker
```
<searchComponent name="spellcheck" class="solr.SpellCheckComponent">
  <lst name="spellchecker">
    <str name="name">default</str>
    <str name="field">name</str>
    <str name="classname">solr.DirectSolrSpellChecker</str>
    <str name="distanceMeasure">internal</str>
    <float name="accuracy">0.5</float>
    <int name="maxEdits">2</int>
    <int name="minPrefix">1</int>
    <int name="maxInspections">5</int>
    <int name="minQueryLength">4</int>
    <float name="maxQueryFrequency">0.01</float>
    <float name="thresholdTokenFrequency">.01</float>
  </lst>
</searchComponent>
```

Add to request to use spellcheck
```
<str name="spellcheck">true</str>
```

## Add custom field type for spellcheck (So to not pollute the query results)
```
<fieldType name="text_spell" class="solr.TextField" positionIncrementGap="100">
  <analyzer>
    <tokenizer class="solr.StandardTokenizerFactory"/>
    <charFilter class="solr.PatternReplaceCharFilterFactory" pattern="[^a-zA-Z0-9]" replacement="" />
    <filter class="solr.LowerCaseFilterFactory"/>
  </analyzer>
</fieldType>
```

Add to schema
```
<field name="spell" type="text_spell" indexed="true" stored="false" multiValued="true"/>
```

In [114]:
# Add custom field type for spellcheck
!curl -X POST -H "Content-type:application/json" --data-binary "{\"add-field-type\":{\"name\":\"text_spell\",\"class\":\"solr.TextField\",\"positionIncrementGap\":100,\"analyzer\":{\"tokenizer\":{\"class\":\"solr.StandardTokenizerFactory\"},\"charFilters\":[{\"class\":\"solr.PatternReplaceCharFilterFactory\",\"pattern\":\"[^a-zA-Z0-9]\",\"replacement\":\"\"}],\"filters\":[{\"class\":\"solr.LowerCaseFilterFactory\"}]}}}" http://localhost:8983/api/cores/search_reddit/schema


# # Add to schema field
!curl -X POST -H "Content-type:application/json" --data-binary "{\"add-field\":{\"name\":\"spell\",\"type\":\"text_spell\",\"indexed\":true,\"stored\":false,\"multiValued\":true}}" http://localhost:8983/api/cores/search_reddit/schema

{
  "responseHeader":{
    "status":0,
    "QTime":97
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   158  100    59  100    99    190    319 --:--:-- --:--:-- --:--:--   509
100   158  100    59  100    99    190    319 --:--:-- --:--:-- --:--:--   509


Thus, our spellchecker will be

```
  <searchComponent name="spellcheck" class="solr.SpellCheckComponent">

    <str name="queryAnalyzerFieldType">text_reddit</str>

    <!-- Multiple "Spell Checkers" can be declared and used by this
         component
      -->

    <!-- a spellchecker built from a field of the main index -->
    <lst name="spellchecker">
      <str name="name">default</str>
      <str name="field">text_spellcheck</str>
      <str name="classname">solr.DirectSolrSpellChecker</str>
      <!-- the spellcheck distance measure used, the default is the internal levenshtein -->
      <str name="distanceMeasure">internal</str>
      <!-- minimum accuracy needed to be considered a valid spellcheck suggestion -->
      <float name="accuracy">0.5</float>
      <!-- the maximum #edits we consider when enumerating terms: can be 1 or 2 -->
      <int name="maxEdits">2</int>
      <!-- the minimum shared prefix when enumerating terms -->
      <int name="minPrefix">1</int>
      <!-- maximum number of inspections per result. -->
      <int name="maxInspections">5</int>
      <!-- minimum length of a query term to be considered for correction -->
      <int name="minQueryLength">4</int>
      <!-- maximum threshold of documents a query term can appear to be considered for correction -->
      <float name="maxQueryFrequency">0.01</float>
      <!-- uncomment this to require suggestions to occur in 1% of the documents
        <float name="thresholdTokenFrequency">.01</float>
      -->
    </lst>
```

# Restart core to apply changes
## Somehow http request for this dosent work, manually edit solr-config.xml and refresh core

In [115]:
# # Edit spellchecker to work for text_reddit, the custom type we created
# !curl -X POST -H "Content-type: application/json" --data-binary "{"add-searchcomponent": {"name": "spellcheck", "class": "solr.SpellCheckComponent", "config": {"queryAnalyzerFieldType": "text_reddit", "spellchecker": {"name": "default", "field": "text_spellcheck", "classname": "solr.DirectSolrSpellChecker", "distanceMeasure": "internal", "accuracy": 0.5, "maxEdits": 2, "minPrefix": 1, "maxInspections": 5, "minQueryLength": 4, "maxQueryFrequency": 0.01}}}}" http://localhost:8983/api/cores/search_reddit/config

## Refresh core

In [124]:
!curl -X POST "http://localhost:8983/solr/admin/cores?action=RELOAD&core=search_reddit&wt=json"

{
  "responseHeader":{
    "status":0,
    "QTime":331
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    60  100    60    0     0    109      0 --:--:-- --:--:-- --:--:--   109


# Ingest data

In [117]:
# add document to solr
!curl "http://localhost:8983/solr/search_reddit/update?commit=true" --data-binary @data/cleaned_combined_data.csv -H "Content-type:application/csv"

{
  "responseHeader":{
    "status":0,
    "QTime":7818
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 25 20.0M    0     0   25 5312k      0  5459k  0:00:03 --:--:--  0:00:03 5459k
 40 20.0M    0     0   40 8320k      0  5100k  0:00:04  0:00:01  0:00:03 5098k
 64 20.0M    0     0   64 12.8M      0  4193k  0:00:04  0:00:03  0:00:01 4194k
 83 20.0M    0     0   83 16.8M      0  4396k  0:00:04  0:00:03  0:00:01 4396k
 96 20.0M    0     0   96 19.2M      0  3989k  0:00:05  0:00:04  0:00:01 3989k
100 20.0M    0     0  100 20.0M      0  3442k  0:00:05  0:00:05 --:--:-- 3048k
100 20.0M    0     0  100 20.0M      0  2944k  0:00:06  0:00:06 --:--:-- 2284k
100 20.0M    0     0  100 20.0M      0  2573k  0:00:07  0:00:07 --:--:-- 1516k
100 20.0M  100    61  100 20.0M      7  2554k  0:00:08  0:00:08 --:--:--  798k


# Test query data

- https://solr.apache.org/guide/6_6/the-standard-query-parser.html

## Select with keyword search

In [118]:
!curl -X GET http://localhost:8983/solr/search_reddit/query -d "q=text:congrats*&rows=10"

{
  "responseHeader":{
    "status":0,
    "QTime":3,
    "params":{
      "q":"text:congrats*",
      "rows":"10"
    }
  },
  "response":{
    "numFound":561,
    "start":0,
    "numFoundExact":true,
    "docs":[{
      "author":"pboksz",
      "text":["Congrats! That is one impressive reward! This is such a fun car and it will be great for hauling the two littles one!"],
      "created_utc":"2022-03-29T03:02:07Z",
      "id":"i2h052j",
      "permalink":"/r/BMWi3/comments/tqghan/my_reward_for_finally_finishing_my_bachelors/i2h052j/",
      "upvote":[10],
      "subreddit_name":"BMWi3",
      "type":"comment",
      "post_id":"t3_tqghan",
      "text_spellcheck":["Congrats! That is one impressive reward! This is such a fun car and it will be great for hauling the two littles one!"],
      "_version_":1795491665914363905
    },{
      "author":"jazzcat007",
      "text":["Youâ€™re gonna love it! Congrats!"],
      "created_utc":"2022-03-29T07:48:17Z",
      "id":"i2i2ebt",
      "perm

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  5411  100  5387  100    24  24040    107 --:--:-- --:--:-- --:--:-- 24156


## Search with timeframe range filter

In [119]:
!curl -X GET http://localhost:8983/solr/search_reddit/query -d "q=created_utc:[2017-01-01T00:00:00Z TO 2018-01-31T23:59:59Z]&rows=10"

{
  "responseHeader":{
    "status":0,
    "QTime":2,
    "params":{
      "q":"created_utc:[2017-01-01T00:00:00Z TO 2018-01-31T23:59:59Z]",
      "rows":"10"
    }
  },
  "response":{
    "numFound":1923,
    "start":0,
    "numFoundExact":true,
    "docs":[{
      "author":"billbucket",
      "text":["Tesla's Summon feature was very useful today..."],
      "created_utc":"2018-01-25T07:13:35Z",
      "id":"7srdiw",
      "num_comments":[1050.0],
      "permalink":"/r/teslamotors/comments/7srdiw/teslas_summon_feature_was_very_useful_today/",
      "upvote":[49182],
      "subreddit_name":"teslamotors",
      "upvote_ratio":[0.9],
      "url":"https://gfycat.com/ReliableSecretJunebug",
      "type":"post",
      "text_spellcheck":["Tesla's Summon feature was very useful today..."],
      "_version_":1795491665902829573
    },{
      "author":"TheMightyCraken",
      "text":["Elon Musk confirms secret Tesla â€˜Carless Driverâ€™ project"],
      "created_utc":"2018-01-30T05:20:51Z",
    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  5958  100  5890  100    68  27446    316 --:--:-- --:--:-- --:--:-- 27841


## Combination of conditions

In [120]:
!curl -X GET http://localhost:8983/solr/search_reddit/query -d "q=text:congrats* AND created_utc:[2017-01-01T00:00:00Z TO 2018-01-31T23:59:59Z]&rows=10"

{
  "responseHeader":{
    "status":0,
    "QTime":4,
    "params":{
      "q":"text:congrats* AND created_utc:[2017-01-01T00:00:00Z TO 2018-01-31T23:59:59Z]",
      "rows":"10"
    }
  },
  "response":{
    "numFound":4,
    "start":0,
    "numFoundExact":true,
    "docs":[{
      "author":"the8king",
      "text":["Wow, congrats!! I wonder how much that was vs a gas roadtrip!"],
      "created_utc":"2017-03-28T23:00:50Z",
      "id":"dfijlrq",
      "permalink":"/r/teslamotors/comments/61yrt5/we_just_drove_from_san_francisco_to_new_york_city/dfijlrq/",
      "upvote":[2],
      "subreddit_name":"teslamotors",
      "type":"comment",
      "post_id":"t3_61yrt5",
      "text_spellcheck":["Wow, congrats!! I wonder how much that was vs a gas roadtrip!"],
      "_version_":1795491671741300746
    },{
      "author":"Randomd0g",
      "text":["You made it! Congrats, friends. About what percentage of the total drive would you say was done with AP?"],
      "created_utc":"2017-03-28T21:10:44

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  2926  100  2839  100    87  13472    412 --:--:-- --:--:-- --:--:-- 13933


## Test spellchecker

In [125]:
!curl -X POST "http://localhost:8983/solr/admin/cores?action=RELOAD&core=search_reddit&wt=json"

{
  "responseHeader":{
    "status":0,
    "QTime":143
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    60  100    60    0     0    167      0 --:--:-- --:--:-- --:--:--   168


In [126]:
!curl "http://localhost:8983/solr/search_reddit/spell?indent=true&spellcheck.q=Tesle&spellcheck=true&spellcheck.collate=true"

{
  "responseHeader":{
    "status":0,
    "QTime":4
  },
  "response":{
    "numFound":0,
    "start":0,
    "numFoundExact":true,
    "docs":[ ]
  },
  "spellcheck":{
    "suggestions":["tesle",{
      "numFound":5,
      "startOffset":0,
      "endOffset":5,
      "origFreq":1,
      "suggestion":[{
        "word":"tesla",
        "freq":4719
      },{
        "word":"tesls",
        "freq":2
      },{
        "word":"teslÃ¥",
        "freq":1
      },{
        "word":"teslas",
        "freq":465
      },{
        "word":"title",
        "freq":74
      }]
    }],
    "correctlySpelled":false,
    "collations":[ ]
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   630  100   630    0     0   2855      0 --:--:-- --:--:-- --:--:--  2850
